In [72]:
import os
import numpy as np
import pandas as pd

root="../res/koges"
variableTypeMapper={
    "":"",
    "연속형":"contiguous",
    "범주형":"sparse",
}

def _view(elem):
    if isinstance(elem,list):
        next={}
        for pair in tuple(q.split("=") for q in elem):
            if len(pair)==2:
                next[int(pair[0])]=pair[1]
        return next
    return elem

In [73]:
x=pd.read_csv(os.path.join(root,"base_data1.txt"))

In [74]:
koges=pd.concat(
    [q.set_index(["T_ID","T_DATA_CLASS"]) for q in [pd.read_csv(os.path.join(root,q.name)) for q in os.scandir(root) if q.is_file() and q.name.startswith("BASE")]],
    axis=1
)

In [75]:
koges.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10000 entries, ('K_BASE_00001', 'B20') to ('K_BASE_10000', 'B11')
Data columns (total 59 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   T_EDATE     10000 non-null  int64  
 1   T_SEX       10000 non-null  int64  
 2   T_AGE       10000 non-null  int64  
 3   T_INCOME    10000 non-null  int64  
 4   T_MARRY     10000 non-null  int64  
 5   T_HTN       10000 non-null  int64  
 6   T_HTNAG     10000 non-null  int64  
 7   T_DM        10000 non-null  int64  
 8   T_DMAG      10000 non-null  int64  
 9   T_LIP       10000 non-null  int64  
 10  T_LIPAG     10000 non-null  int64  
 11  T_FMFHT1    10000 non-null  int64  
 12  T_FMFHT2    10000 non-null  int64  
 13  T_FMFDM1    10000 non-null  int64  
 14  T_FMFDM2    10000 non-null  int64  
 15  T_DRINK     10000 non-null  int64  
 16  T_DRDU      10000 non-null  int64  
 17  T_TAKFQ     10000 non-null  int64  
 18  T_TAKAM     10000 non-null

In [76]:
label=pd.read_excel(
    os.path.join(root,"[기반조사]_KoGES_코호트_자료_분석_코드북.xlsx"),
    skiprows=2
).set_axis(
    ["id","table","table_","variable","variableLabel","variableInfo","variableType","variableView"],
    axis=1
)

# Excel preprocessing
labelView=label.map(
    lambda q:"" if pd.isna(q) else q
)
labelView.loc[:,"variableInfo"]=labelView.variableInfo.map(
    lambda q:q[q.find("\n")+1:]
)
labelView.loc[:,"variableInfo"]=labelView.variableInfo.map(
    lambda q:q[:q.rfind("\n")]
)
labelView.loc[:,"variableInfo"]=labelView.variableInfo.map(
    lambda q:q.replace("\n",", ") if isinstance(q,str) else q
)
labelView.loc[:,"variableInfo"]=labelView.variableInfo.apply(
    lambda q:q.split(", ") if q.__contains__(", ") else q
)
labelView.loc[:,"variableInfo"]=labelView.variableInfo.map(
    _view
)
labelView.loc[:,"variableType"]=list(map(
    lambda q:variableTypeMapper[q],labelView.variableType
))

In [77]:
# labelView=labelView.set_index("variable").to_dict(orient="index")
print("Variable(s) missing in the book","\n",
    [q for q in koges.columns if not q in labelView.variable.unique()]
)

Variable(s) missing in the book 
 ['T_TOTALC', 'T_PACKYR']


In [78]:
koges.select_dtypes("number").agg(["mean","median","std","max","min"])

,T_EDATE,T_SEX,T_AGE,T_INCOME,T_MARRY,T_HTN,T_HTNAG,T_DM,T_DMAG,T_LIP,...,T_BMI,T_HBA1C,T_GLU0,T_CREATINE,T_AST,T_ALT,T_TCHL,T_HDL,T_LDL,T_TG
mean,200703.76220,1.654000,53.705700,17213.643600,702.24470,181.196900,62932.949100,191.067400,72535.066100,221.074800,...,533.890200,44685.033440,2452.982400,40.834976,2393.803600,2582.14160,247.163200,92.806700,2077.15600,389.006600
median,200706.00000,2.000000,53.000000,6.000000,2.00000,1.000000,77777.000000,1.000000,77777.000000,1.000000,...,24.000000,66666.000000,91.000000,0.800000,22.000000,19.00000,195.000000,51.000000,118.00000,107.000000
std,349.33573,0.475717,8.722666,35082.517497,8337.41024,4238.939271,30871.250374,4354.883576,19742.420575,4685.390321,...,7121.769159,35970.361833,15166.095429,1999.663253,15208.192649,15790.92508,2231.474204,1998.666128,13846.14353,5086.790788
max,201312.00000,2.000000,86.000000,99999.000000,99999.00000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,...,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.00000,99999.000000,99999.000000,99999.00000,99999.000000
min,200101.00000,1.000000,40.000000,1.000000,1.00000,1.000000,10.000000,1.000000,27.000000,1.000000,...,14.000000,3.200000,49.000000,0.310000,5.000000,6.00000,76.000000,18.000000,2.00000,26.000000


In [79]:
koges.shape

(10000, 59)

In [80]:
sieve=(((koges.reset_index().set_index("T_ID").select_dtypes("number")).ge(60000)).apply(sum,axis=1)).gt(18)

In [81]:
koges.iloc[sieve.values,:].sample(5)

,,T_EDATE,T_SEX,T_AGE,T_INCOME,T_MARRY,T_HTN,T_HTNAG,T_DM,T_DMAG,T_LIP,...,T_BMI,T_HBA1C,T_GLU0,T_CREATINE,T_AST,T_ALT,T_TCHL,T_HDL,T_LDL,T_TG
T_ID,T_DATA_CLASS,,,,,,,,,,,,,,,,,,,,,
K_BASE_06671,B14,200705,1,66,5,2,2,56,1,77777,1,...,22,66666.0,99999,1.00,99999,99999,201,79,109,63
K_BASE_09652,B32,200411,1,70,66666,2,1,77777,1,77777,1,...,19,66666.0,105,1.06,23,14,169,58,90,105
K_BASE_03282,B01,200308,1,59,7,2,1,77777,1,77777,1,...,22,66666.0,95,1.10,21,17,211,58,140,64
K_BASE_01581,B15,200404,1,58,66666,2,1,77777,1,77777,1,...,23,66666.0,86,1.02,19,17,155,43,103,46
K_BASE_07480,B20,200807,1,58,7,2,1,77777,2,99999,1,...,23,66666.0,108,1.10,19,20,170,56,96,91


In [82]:
(((koges.reset_index().set_index("T_ID").select_dtypes("number")).ge(60000)).apply(sum)).describe()

count       59.000000
mean      2775.864407
std       3732.460358
min          0.000000
25%         19.500000
50%        139.000000
75%       6262.500000
max      10000.000000
dtype: float64

In [83]:
sieveCol=((koges.reset_index().set_index("T_ID").select_dtypes("number")).ge(60000)).apply(sum).ge(1600).values

In [84]:
koges.loc[:,sieveCol]

,,T_EDATE,T_INCOME,T_HTNAG,T_DMAG,T_LIPAG,T_DRDU,T_TAKAM,T_RICEAM,T_WINEAM,T_SOJUAM,...,T_SMDUYR,T_SMDUMO,T_SMAM,T_PACKYR,T_MNSAG,T_PMYN,T_PMAG,T_PREG,T_FPREGAG,T_HBA1C
T_ID,T_DATA_CLASS,,,,,,,,,,,,,,,,,,,,,
K_BASE_00001,B20,200802,99999,77777,77777,42,77777,77777.0,77777.0,77777.0,77777.0,...,77777.0,66666,77777.0,77777.0,99999,1,77777,2,22,66666.0
K_BASE_00002,B20,200306,1,50,77777,77777,3,77777.0,77777.0,77777.0,77777.0,...,77777.0,66666,77777.0,77777.0,16,2,55,2,18,66666.0
K_BASE_00003,B14,201108,99999,50,77777,50,77777,77777.0,77777.0,77777.0,77777.0,...,77777.0,77777,77777.0,77777.0,14,2,47,2,27,66666.0
K_BASE_00004,B29,201004,1,63,77777,77777,77777,77777.0,77777.0,77777.0,77777.0,...,77777.0,77777,77777.0,77777.0,16,2,54,2,21,66666.0
K_BASE_00005,B23,200410,2,77777,77777,77777,77777,77777.0,77777.0,77777.0,77777.0,...,77777.0,77777,77777.0,77777.0,14,2,50,2,24,66666.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
K_BASE_09996,B20,201003,3,77777,77777,77777,99999,77777.0,77777.0,77777.0,77777.0,...,77777.0,66666,77777.0,77777.0,77777,77777,77777,77777,77777,66666.0
K_BASE_09997,B20,200510,7,58,48,77777,4,77777.0,77777.0,77777.0,77777.0,...,40.0,66666,20.0,99999.0,77777,77777,77777,77777,77777,66666.0
K_BASE_09998,B14,201107,5,77777,77777,77777,4,77777.0,77777.0,77777.0,6.5,...,35.0,0,10.0,17.5,77777,77777,77777,77777,77777,66666.0


In [93]:
ukValueMapper={
    66666:np.nan,
    77777:None,
    99999:np.nan
}
koges.map(lambda q:ukValueMapper[q] if q in (66666,77777,99999) else q).agg(lambda q:pd.notna(q).value_counts(normalize=True)).T*100

,False,True
T_EDATE,NaN,100.00
T_SEX,NaN,100.00
T_AGE,NaN,100.00
T_INCOME,20.12,79.88
T_MARRY,0.70,99.30
T_HTN,0.18,99.82
T_HTNAG,80.65,19.35
T_DM,0.19,99.81
T_DMAG,93.13,6.87
T_LIP,0.22,99.78


In [103]:
from sklearn.linear_model import BayesianRidge
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


Eq=BayesianRidge(tol=tuple(1e-1*(q) for q in (np.logspace(1,1.7,4)))
)
Imputer=IterativeImputer(Eq)

Imputer.fit

<bound method IterativeImputer.fit of IterativeImputer(estimator=BayesianRidge(tol=(1.0, 1.711328304161781,
                                              2.9286445646252357,
                                              5.011872336272723)))>